#### Job Shop Scheduling ####

**Short summary**

1. The problem consists of N *jobs* 1..N  each job has some number of *tasks*
1. For each job, the tasks must be done in sequence
1. Each task requires a *resource*,  and two tasks that use the same resource cannot be scheduled at the same time
1. All tasks take three time units to complete
1. All tasks are ready for execution at time 0, and must complete at or before time 15
  1. Which means that if any job has more five tasks, it cannot be scheduled, no matter what

There are some hints at the bottom of the file.

Input to the problem comes in a variable like this:


In [ ]:
# Job 1
#   (o11, r1)   (o12, r2) (o13, r3)
# Job 2
#   (o21, r1)  (o22, r2)
# Job 3
#   (o31, r3) (o32, r1) (o33, r2)
# Job 4
#   (o41, r4) (o42, r2)
#
# The ri are the resources

example = [[("o11", "r1"), ("o12", "r2"), ("o13", "r3")],
           [("o21", "r1"), ("o22", "r2")],
           [("o31", "r3"), ("o32", "r1"), ("o33", "r2")],
           [("o41", "r4"), ("o42", "r2")]]


----------------------------------------------

In [ ]:
from constraint import *

In [ ]:
###  Your implementation of solveCSP here:  takes a problem description as input, returns a 
###  list of solutions as output.  The method problem.getSolutions() returns a list of solutions.
###  Each solution is a dictionary -- key is a task, value is the value it is assigned by the CSP
def subtasks(problemList, jobno): # return a subtask list, ex: ['o11', 'o12', 'o13']
    subTaskList = []
    for job in problemList[jobno]:
        subTaskList.append(job[0])
    return subTaskList

def resourceFor(subtask, problemList): # return a resource by inputting the given subtask, ex: 'r1'
    for job in problemList:
        for subTask in job:
            if subtask == subTask[0]:
                return subTask[1]

def addOrderConstraints(problemList, problem): # add order constraints
    for job in problemList:
        for i in range(len(job)-1):
            if len(job) > 1:
                problem.addConstraint(lambda x, y: x + 1 <= y, [job[i][0], job[i+1][0]])
                
def addMutexConstraints(problemList, problem):
    resList = [] # generate the resource list, ex:['r1', 'r2', 'r3', 'r4']
    for job in problemList:
        for subtask in job:
            if subtask[1] not in resList:
                resList.append(subtask[1])
    
    resTaskList = [] # create the task list ordered by resource ['r1', 'r2', 'r3', 'r4']                 
    for res in resList: # ex:[['o11', 'o21', 'o32'], ['o12', 'o22', 'o33', 'o42'], ['o13', 'o31'], ['o41']]
        taskList = []
        for job in problemList:
            for subtask in job:
                if subtask[1] == res:
                     taskList.append(subtask[0])
        resTaskList.append(taskList) 
    
    for tasks in resTaskList: # find every combination for every resource task list and set it different
        size = len(tasks)     # ex: for r1, the combinations are [(o11,o21),(o11, o32),(o21,o32)] 
        for i in reversed(range(1,size)):
            for j in reversed(range(i)):
                problem.addConstraint(lambda x, y: x != y, [tasks[size-1-i], tasks[size-1-j]])
                    
                
def solveCSP(problemList): 
    problem = Problem()
    variables = []
    
    for job in problemList:
        for subTask in job:
            variables.append(subTask[0]) # create a variable list, ex: [o11, o12, o13...]
            
    problem.addVariables(variables, range(0,5))
    addOrderConstraints(problemList, problem)      
    addMutexConstraints(problemList, problem)
    
    return problem.getSolutions()

In [ ]:
def printSolution(problemList):
    solutions = solveCSP(problemList)
    if len(solutions) == 0:
        print("No solutions for this problem")
    else:
        print("There are " + str(len(solutions)) + " solutions.  Here is one:\n")
        solution = solutions[0]
        for jobno in range(0, len(problemList)):
            print("Schedule for Job " + str(jobno+1))
            for subtask in sorted(subtasks(problemList, jobno)):
                print("   Subtask " + subtask + " starts at " + str(solution[subtask]*3) + \
                      " ends at " + str(2 + int(solution[subtask]*3)) + "\tuses " + resourceFor(subtask, problemList))
                

In [ ]:
printSolution(example)

-----------------------------------

**Hints**
1.  Since all tasks take three time units, it's easiest to partition time into "buckets" each of which is three time units long.  For example, bucket 1 is the interval between t=0 and t=2 inclusive, bucket 1 is t=3 to t=5 inclusive.  You will notice from the **printSolution** code that my solver does that bucketing, then when it prints the solution it converts the time bucket number to a real point in time
2.  There are only two kinds of constraints:  order constraints (one task must come after another) and mutex constraints (two tasks must not be scheduled at the same time).   If you write helper functions for those two constraints, it will simplify your code a lot.